### WE WILL LOAD BREAST CANCER DATA IN AI DB 
### DECIMALIZE IT
### SEPERATE TRAINING (80%) AND TESTING (20%) DATA AND LOAD IN CSV
### RUN DIFFERENT AI'S ON IT



In [2]:
from sklearn.datasets import *
from sqlalchemy import create_engine
import numpy
import pandas
from pandas import *
import pandas.io.sql as psql
import csv
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib
import psycopg2
import glob, os
from collections import OrderedDict
import shutil
import pdb
from pdb import set_trace as bp
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegression

In [3]:
fp=r'/Users/Aseem/breast_cancer/'
no_of_partitions_per_para=10
boston = load_boston()
breast = load_breast_cancer()
print(boston.data.shape)
print(breast.data.shape)

(506, 13)
(569, 30)


In [4]:
# #GETTING BREAST CANCER RAW DATA IN APPROPRIATE FORMAT IN A CSV FILE

# with open(fp+'breast_cancer_data.csv', 'w', newline='\n') as fp:  # NEW CSV FILE IS CREATED
#     csvfile = csv.writer(fp, delimiter=',')
#     headers=[]
#     headers.append('id') #WE HAVE TO GIVE UNIQUE IDS OURSELF HERE
#     for v in list(breast['feature_names']):
#         headers.append(v)
#     headers.append('class')
#     csvfile.writerow(headers)
#     data_1=[list(x) for x in breast['data']]
#     data_2=[]
#     for i in range (0,len(data_1)):
#         x=[]
#         x.append(i)
#         for v in data_1[i]:
#             x.append(v) 

#         x.append(breast['target'][i])
#         data_2.append(x)        
#         csvfile.writerow(x)
        

In [5]:
# #GENERATING DECIMALIZED TRAIN AND TEST CSV FOR BREAST CANCER

conn=psycopg2.connect(database="AI_DB_LOCAL",user="Aseem",host="localhost")
cur = conn.cursor()

In [6]:
# PROC_BREAST_CANCER_DATA_IP=[no_of_partitions_per_para,fp]
# cur.callproc('PROC_BREAST_CANCER_DATA',PROC_BREAST_CANCER_DATA_IP)
# conn.commit() #commiting is important

In [16]:
#RANDOM FOREST BREAST CANCER DECIMALIZED TRAIN & TEST. ACCURACY FLUCTUATES FROM 91% TO 96%
df_train=pandas.read_csv(fp+'Train_1.csv')
df_test=pandas.read_csv(fp+'Test_1.csv')
features=df_train.columns[1:-1] #taking specific columns for training.ignoring ID and class column 

clf = RandomForestClassifier(n_estimators=350,n_jobs=2, random_state=0)# Create a random forest Classifier. By convention, clf means 'Classifier'

clf.fit(df_train[features], df_train['class']) #Training Random Forest . clf.fit(data,target)
rf_pred=clf.predict(df_test[features])

df_test_prob=pandas.DataFrame(np.column_stack([rf_pred,df_test['id']]),columns=['rf_pred','id'])

engine = create_engine('postgresql+psycopg2://Aseem:@localhost/AI_DB_LOCAL')

df_test_prob.to_sql('breast_cancer_rf_test_prob', engine,schema='internal',if_exists='replace',index=False)

rf_analysis=psql.read_sql("""
    select sum(accuracy) sum_of_accuracy,round((sum(accuracy)::numeric/count(*))*100,2) as total_accuracy_perc
    from(
        select a.*,b.class,case when a.rf_pred=b.class then 1 else 0 end as accuracy 
        from internal.breast_cancer_rf_test_prob a
        left join internal.breast_cancer_test b
        on a.id=b.id
        )a;"""
           ,conn  )
conn.commit()

print('Random Forest Accuracy\n',rf_analysis)

Random Forest Accuracy
    sum_of_accuracy  total_accuracy_perc
0              102                90.27


In [24]:
#RANDOM FOREST BREAST CANCER RAW DATA  #ACCURACY FLUCTUATES FROM 91% TO 96%
breast = load_breast_cancer()
df = pandas.DataFrame(breast.data, columns=breast.feature_names)

df['species'] = pandas.Categorical.from_codes(breast.target, breast.target_names)
df['is_train'] = numpy.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
df.head()
features=breast.feature_names
y = pandas.factorize(train['species'])[0]

clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(train[features], y)

preds = breast.target_names[clf.predict(test[features])]
accuracy=0
for i in range (0,preds.shape[0]):
    if test['species'].values[i]==preds[i]:
        accuracy=accuracy+1
print('total_accuracy_perc=',(accuracy/preds.shape[0])*100)
# Create confusion matrix
pandas.crosstab(test['species'], preds, rownames=['Actual Species'], colnames=['Predicted Species'])

total_accuracy_perc= 95.19999999999999


Predicted Species,benign,malignant
Actual Species,,
malignant,2,46
benign,73,4


In [4]:
#RANDOM FOREST 2
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestClassifier(n_estimators=150, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred, target_names=['Cancer', 'No Cancer']))
print(f'Overall Accuracy: {metrics.accuracy_score(y_test, y_pred)*100:0.2f}%')

             precision    recall  f1-score   support

     Cancer       0.94      0.98      0.96        47
  No Cancer       0.98      0.96      0.97        67

avg / total       0.97      0.96      0.97       114

Overall Accuracy: 96.49%


In [17]:
#NIV2  conf_thresh=[1,0.9,0.8]  accuracy= [95.58%,93.81%,93.81%] 

df_test_prob_NI=pandas.read_csv(fp+'Probability/Test_1_prob.csv')
df_test_prob2_NI=pandas.DataFrame(np.column_stack([df_test_prob_NI['id'],df_test_prob_NI['NFM_C0'],df_test_prob_NI['NFM_C1']])
                 ,columns=['id','total_nfm_c0','total_nfm_c1'])
df_test_prob2_NI.to_sql('breast_cancer_niv2_test_prob', engine,schema='internal',if_exists='replace',index=False)

niv2_analysis=psql.read_sql("""
    --NIV2
    select sum(accuracy) sum_of_accuracy,round((sum(accuracy)::numeric/count(*))*100,2) as total_accuracy_perc
    from(
        select *,case when pred_class=class then 1 else 0 end as accuracy 
        from(
            select a.*,b.class,CASE when a.Total_NFM_C0>a.Total_NFM_C1 then 0 else 1 end as pred_class
            from internal.breast_cancer_niv2_test_prob a
            left join internal.breast_cancer_test b
            on a.id=b.id
            )a
        )b;"""
         ,conn  )
conn.commit()

print('NIV2 Accuracy\n',niv2_analysis)

NIV2 Accuracy
    sum_of_accuracy  total_accuracy_perc
0              106                93.81


In [147]:
#NEURAL NETWORK --MULTI LAYER PERCEPTRON CLASSIFIER 
train_data, test_data, train_target, test_target = train_test_split(breast.data, breast.target, random_state=5)# SPLITS DATA INTO TRAIN AND TEST
print('no of train samples=',len(train_target))
print('no of test samples=',len(test_target))
mlp = MLPClassifier(random_state=42)
mlp.fit(train_data, train_target)
print('total_accuracy_perc=',round((mlp.score(test_data, test_target))*100,2))

no of train samples= 426
no of test samples= 143
total_accuracy_perc= 89.51


In [149]:
#LOGISTIC REGRESSION - MOST BASIC
train_data, test_data, train_target, test_target = train_test_split(breast.data, breast.target, random_state=100)# SPLITS DATA INTO TRAIN AND TEST
print('no of train samples=',len(train_target))
print('no of test samples=',len(test_target))
log_reg = LogisticRegression()
log_reg.fit(train_data, train_target)
print('total_accuracy_perc=',round((log_reg.score(test_data, test_target))*100,2))
lr_prob=log_reg.predict_proba(test_data)
# numpy.set_printoptions(suppress=True)
# numpy.array(lr_prob,numpy.float32)

no of train samples= 426
no of test samples= 143
total_accuracy_perc= 95.8
